This notebook produces the following figures that are used in the manuscript:

1. Visual convergence of the solution of $\text{KdVH}(\tau)$ to the KdV solution using a time-stepping method.
2. Traveling Waves Using the Petviashvili Method.
3. Quantitative comparison of the asymptotic-preserving (AP) property by different time-stepping methods.
4. Asymptotic Accuracy Plots.
5. Error growth plots for different methods, with and without RK relaxation, applied to $\text{KdVH}(\tau)$, with the exact solution computed using the Petviashvili method.
6. RK relaxation on $\text{KdVH}(\tau)$ to verify linear and quadratic error convergence using conservative versus non-conservative methods.

In [ ]:
include("code.jl")

In [ ]:
using CairoMakie
set_theme!(theme_latexfonts();
           fontsize = 20,
           Lines = (linewidth = 3, cycle = Cycle([:color, :linestyle], covary = true),),
           markersize = 16)

### Visual convergence of the solution of $\text{KdVH}(\tau)$ to the KdV solution.

In [ ]:
domain_traversals = 1.2; # This will determine the final time = domain_traversals*(xmax-xmin)/c
accuracy_order = 1; N = 2^8;
alg = ARS443(); dt = 0.01;
τs = [1.0e-1, 5.0e-2, 1.0e-2]

# Initialization of physical and numerical parameters
(; xmin, xmax, c) = solitary_wave_setup()
tspan = (0.0, domain_traversals * (xmax - xmin) / c)

D1 = upwind_operators(periodic_derivative_operator;
                      derivative_order = 1, accuracy_order,
                      xmin, xmax, N)


# Setup plot
fig_KdVH_KdV = Figure()
ax = Axis(fig_KdVH_KdV[1, 1]; xlabel = L"x", ylabel = L"u")

# KdV
let equation = KdV()
    (; q0, parameters) = setup(solitary_wave_solution, equation,
                               tspan, D1)
    @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                           rhs_nonstiff!,
                           q0, tspan, parameters, alg;
                           dt)

    x = grid(parameters.D1)
    lines!(ax, x, get_u(sol.u[begin], equation);
           label = L"u_0(x)", color = :gray)
    lines!(ax, x, get_u(sol.u[end], equation);
           label = L"u_\mathrm{KdV}", color = :black)
end

# Assuming `τs` is an array of different values of τ=
for (i, τ) in enumerate(τs)
    # HyperbolizedKdV with IMEX
    let equation = HyperbolizedKdV(τ)
        (; q0, parameters) = setup(solitary_wave_solution, equation,
                                   tspan, D1)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt)

        x = grid(parameters.D1)
        # Use label with τ and different line styles
        lines!(ax, x, get_u(sol.u[end], equation);
               label = L"u_\mathrm{KdVH}: \tau = %$(τ)")
    end
end

axislegend(ax, position = :lt)
xlims!(ax, xmin, xmax)
fig_KdVH_KdV

In [ ]:
# Define paths
save_path_1 = joinpath(@__DIR__, "img")
save_path_2 = joinpath(@__DIR__, "tables_data")

# Create directories if they do not exist
if !isdir(save_path_1)
    mkdir(save_path_1)
end

if !isdir(save_path_2)
    mkdir(save_path_2)
end

In [ ]:
save(joinpath(save_path_1, "KdVH_convergence_KdV_ARS443.pdf"), fig_KdVH_KdV)

### Traveling Waves Using the Petviashvili Method.

In [ ]:
# Exact solution for the traveling wave solution for the KdV
function kdv_solution(ξ, c)
    A = 3 * c
    K = sqrt(3 * A) / 6

    return A / cosh(K * ξ)^2
end

In [ ]:
function petviashvili_kdv(; xmin = - 40.0, xmax = 40.0, c = 1/3, N = 2^8, maxiter = 1000, tol = 1.0e-14)

    D = fourier_derivative_operator(xmin, xmax, N)

    L = c * I - D^2
    N_fun(u) = 0.5 .* u.^2
    γ = 2
    # Initialization
    x = grid(D)
    u0 = exp.(-x.^2)
    # Iteration
    u = copy(u0)
    for iter in 1:maxiter
        res = norm(L * u - N_fun(u), Inf)
        #println(res)
        if res < tol
            @info "Converged" iter res
            break
        end
        m = integrate(u .* (L * u), D) / integrate(u .* N_fun(u), D)
        u = m^γ * (L \ N_fun(u))
    end

    return (; x, u)
end

In [ ]:
function petviashvili_kdvh(; xmin = - 40.0, xmax = 40.0, λ = 100, c = 1/3, N = 2^8, maxiter = 1000, tol = 1.0e-14)
    # Setup numerical discretization with periodic boundary conditions

    D = fourier_derivative_operator(xmin, xmax, N)
    D1 = Matrix(D);
    L = (λ^2*c/((λ+c)*(λ-c^2))) .* I - D^2

    N_fun(u) = (λ^2/((λ+c)*(λ-c^2))) .*0.5 .* u.^2 + (c/(λ-c^2)) .* D1 * (u .* (D1 * u))
    γ = 2
    # Initialization
    x = grid(D)
    u0 = exp.(-x.^2)

    # Iteration
    u = copy(u0)
    for iter in 1:maxiter
        res = norm(L * u - N_fun(u), Inf)
        #println(res)
        if res < tol
            @info "Converged" iter=iter res=res
            break
        end
        m = integrate(u .* (L * u), D) / integrate(u .* N_fun(u), D)
        u = m^γ * (L \ N_fun(u))
    end
    return (; x, u)
end

#### Plot

In [ ]:
c = 1/3;
x, u_kdv = petviashvili_kdv(; xmin = - 40.0, xmax = 40.0, c = c, N = 2^8, maxiter = 1000, tol = 1.0e-14)
u_ana = kdv_solution.(x, c)

@info "Error wrt KdV solution" norm(u_kdv - u_ana, Inf)

τ1 = 1.0e-0; λ1 = 1/τ1;
x, u_kdvh_1 = petviashvili_kdvh(; xmin = - 40.0, xmax = 40.0, λ = λ1, c = c, N = 2^8, maxiter = 1000, tol = 1.0e-14)
@info "Error wrt KdV solution" norm(u_kdvh_1 - u_ana, Inf)

τ2 = 5.0e-1; λ2 = 1/τ2;
x, u_kdvh_2 = petviashvili_kdvh(; xmin = - 40.0, xmax = 40.0, λ = λ2, c = c, N = 2^8, maxiter = 1000, tol = 1.0e-14)
@info "Error wrt KdV solution" norm(u_kdvh_2 - u_ana, Inf)

τ3 = 1.0e-1; λ3 = 1/τ3;
x, u_kdvh_3 = petviashvili_kdvh(; xmin = - 40.0, xmax = 40.0, λ = λ3, c = c, N = 2^8, maxiter = 1000, tol = 1.0e-14)
@info "Error wrt KdV solution" norm(u_kdvh_3 - u_ana, Inf)

# Visualization
xL = -7*π; xR = 7*π
fig_sol_wave = Figure()
ax = Axis(fig_sol_wave[1, 1]; xlabel = L"x", ylabel = L"u")
lines!(ax, x, u_ana; label = L"u_\mathrm{KdV}", color = :black)
lines!(ax, x, u_kdvh_1; label = L"u_\mathrm{KdVH}: \tau = %$(τ1)")
lines!(ax, x, u_kdvh_2; label = L"u_\mathrm{KdVH}: \tau = %$(τ2)")
lines!(ax, x, u_kdvh_3; label = L"u_\mathrm{KdVH}: \tau = %$(τ3)")
axislegend(ax, position = :rt)
xlims!(ax, xL, xR)
fig_sol_wave

In [ ]:
save(joinpath(save_path_1, "solitary_wave_kdv_kdvh.pdf"), fig_sol_wave)

### Quantitative comparison of the asymptotic-preserving (AP) property by different time-stepping methods.

In [ ]:
# Convergence in the hyperbolic relaxation parameter
function save_table_as_latex(τs, errors_u, eoc_u, errors_v, eoc_v, errors_w, eoc_w, filename)
    data = hcat(τs, errors_u, eoc_u, errors_v, eoc_v, errors_w, eoc_w)
    header = ["\tau", "error u", "EOC u", "error v", "EOC v", "error w", "EOC w"]  # Use LaTeX math mode for τ
    kwargs = (; header, formatters=(ft_printf("%.2e", [1, 2, 4, 6]),
                                    ft_printf("%.2f", [3, 5, 7])))

    open(filename, "w") do io
        println(io, "\\begin{table}[ht]")  # Start the table environment
        println(io, "\\centering")         # Center the table
        pretty_table(io, data; kwargs..., backend=Val(:latex))
        println(io, "\\caption{....}")  # Optional caption
        println(io, "\\end{table}")     # End the table environment
    end
end

function get_uvw(q, equations::HyperbolizedKdV)
    N = length(q) ÷ 3
    u = view(q, (0 * N + 1):(1 * N))
    v = view(q, (1 * N + 1):(2 * N))
    w = view(q, (2 * N + 1):(3 * N))
    return (; u = u,v = v, w = w)
end

function AP_property_imex(; latex = false,
                            save_latex_table = true,
                            domain_traversals = 0.25,
                            N = 2^8,
                            alg = ARS443(),
                            ImEx_name = "ARS443",
                            dt = 0.005,
                            kwargs...)
    # Initialization of physical and numerical parameters
    (; xmin, xmax, c) = solitary_wave_setup()
    tspan = (0.0, domain_traversals * (xmax - xmin) / c)

    D1 = upwind_operators(periodic_derivative_operator;
                          derivative_order = 1, accuracy_order = 8,
                          xmin, xmax, N)


    τs = [1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5, 1.0e-6, 1.0e-7,
         1.0e-8, 1.0e-9, 1.0e-10, 1.0e-11, 1.0e-12]

    errors_u_num = Float64[]
    errors_v_num = Float64[]
    errors_w_num = Float64[]

    u_ref_num = let
        equation = KdV()
        (; q0, parameters) = setup(solitary_wave_solution, equation,
                                   tspan, D1)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt = dt, kwargs...)
        get_u(sol.u[end], equation)
    end

    # Computing the numerical reference solution for the auxiliary variables using the upwind derivative operator
    D = D1.central
    v_ref_num = similar(u_ref_num)
    w_ref_num = similar(u_ref_num)
    mul!(v_ref_num, D, u_ref_num)
    mul!(w_ref_num, D, v_ref_num)

    for τ in τs
        equation = HyperbolizedKdV(τ)
        (; q0, parameters) = setup(solitary_wave_solution, equation,
                                   tspan, D1)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt = dt, kwargs...)

        x = grid(parameters.D1)
        uvw = get_uvw(sol.u[end], equation)

        error_u = integrate(abs2, uvw.u - u_ref_num, parameters.D1) |> sqrt
        error_v = integrate(abs2, uvw.v - v_ref_num, parameters.D1) |> sqrt
        error_w = integrate(abs2, uvw.w - w_ref_num, parameters.D1) |> sqrt
        push!(errors_u_num, error_u); push!(errors_v_num, error_v); push!(errors_w_num, error_w)
    end

    let
        errors_u = errors_u_num
        errors_v = errors_v_num
        errors_w = errors_w_num
        @info "Errors with respect to the numerical KdV solution"
        eoc_u = compute_eoc(inv.(τs), errors_u)
        eoc_v = compute_eoc(inv.(τs), errors_v)
        eoc_w = compute_eoc(inv.(τs), errors_w)

        # Change the name of the file here when use different time-stepping methods
        filename = joinpath(save_path_2, ImEx_name * "_errors_num.tex")
        #filename = "Figures/" * ImEx_name * "_errors_num.tex"
        if save_latex_table
            save_table_as_latex(τs, errors_u, eoc_u, errors_v, eoc_v, errors_w, eoc_w, filename)
        end

        data = hcat(τs, errors_u, eoc_u, errors_v, eoc_v, errors_w, eoc_w)
        header = ["\tau", "error u", "EOC u", "error v", "EOC v", "error w", "EOC w"]  # Use LaTeX math mode for τ
        kwargs = (; header, formatters=(ft_printf("%.2e", [1, 2, 4, 6]),
                                    ft_printf("%.2f", [3, 5, 7])))
        pretty_table(data; kwargs...)
        if latex
            pretty_table(data; kwargs..., backend=Val(:latex))
        end
    end

end

#### AP property by type I methods.

In [ ]:
# AP results by SSP2-IMEX(2,2,2)
#SSP2-IMEX(2,2,2): 2nd order L-stable type I ImEX-RK, explicit part - NOT FSAL, implicit part - NOT SA; NOT GSA.

AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = SSP2ImEx222(),
                   ImEx_name = "SSP2ImEx222",
                   dt = 0.005)

In [ ]:
# AP results by SSP2-IMEX(3,3,2)
# SSP2-IMEX(3,3,2): 2nd order L-stable type I ImEX-RK, explicit part - NOT FSAL, implicit part - SA; NOT GSA.

AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = SSP2ImEx332(),
                   ImEx_name = "SSP2ImEx332",
                   dt = 0.005)

In [ ]:
# AP results by AGSA(3,4,2)
# AGSA(3,4,2): 2nd order type I ImEX-RK, explicit part - FSAL, implicit part - SA; GSA.

AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = AGSA342(),
                   ImEx_name = "AGSA342",
                   dt = 0.005)

In [ ]:
# AP results by SSP3-IMEX(3,4,3)
# SSP3-IMEX(3,4,3): 3rd order L-stable type I ImEX-RK, explicit part - NOT FSAL, implicit part - NOT SA; NOT GSA.

AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = SSP3ImEx343(),
                   ImEx_name = "SSP3ImEx343",
                   dt = 0.005)

#### AP property by type II methods.

In [ ]:
# AP results by ARS(2,2,2)
# ARS(2,2,2): 2nd order L-stable type II ImEX-RK, explicit part - FSAL, implicit part - SA; GSA.
AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = ARS222(),
                   ImEx_name = "ARS222",
                   dt = 0.005)

In [ ]:
# AP results by ARS(4,4,3)
# ARS(4,4,3): 3rd order L-stable type II ImEX-RK, explicit part - FSAL, implicit part - SA; GSA.
AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = ARS443(),
                   ImEx_name = "ARS443",
                   dt = 0.005)

In [ ]:
# AP results by ARK3(2)4L[2]SA–ERK
# ARK3(2)4L[2]SA–ERK: 3rd order type II ImEX-RK, explicit part - NOT FSAL, implicit part - SA; NOT GSA.

AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = ARK324L2SA_ERK(),
                   ImEx_name = "ARK324L2SA_ERK",
                   dt = 0.005)

In [ ]:
# AP results by ARK4(3)6L[2]SA–ERK
# ARK4(3)6L[2]SA–ERK: 4th order type II ImEX-RK, explicit part - NOT FSAL, implicit part - SA; NOT GSA.

AP_property_imex(; latex = false,
                   save_latex_table = true,
                   domain_traversals = 0.25,
                   N = 2^10,
                   alg = ARK436L2SA_ERK(),
                   ImEx_name = "ARK436L2SA_ERK",
                   dt = 0.005)

### Asymptotic Accuracy Plots.

#### Error convergence for the KdVH ($\tau$) system relative to reference solutions computed using a Petviashvili-type algorithm.

In [ ]:
τs = [1.0e-1, 1.0e-4, 1.0e-8, 1.0e-12]
Mthds = [SSP2ImEx222(), SSP3ImEx343(), ARS222(), ARK324L2SA_ERK()]
tols = [5.0e-12, 5.0e-13, 5.0e-13, 5.0e-13]

n_dts = 10; DTs = zeros(4, n_dts)
# Assign each row using broadcasting
DTs[1, :] .= 10 .^ range(log10(2), log10(0.2), length=n_dts)
DTs[2, :] .= 10 .^ range(log10(1.5), log10(0.001), length=n_dts)
DTs[3, :] .= 10 .^ range(log10(1.5), log10(0.001), length=n_dts)
DTs[4, :] .= 10 .^ range(log10(1.5), log10(0.001), length=n_dts)

c = 1/3; domain_traversals = 0.02;
xmin = -40.0; xmax = 40.0; accuracy_order = 8; N = 2^9;

# Initialization of physical and numerical parameters
tspan = (0.0, domain_traversals * (xmax - xmin) / c)
D1 = upwind_operators(periodic_derivative_operator;
   derivative_order = 1, accuracy_order = accuracy_order,
   xmin, xmax, N)

# Initialization of error storage
errors_data = Dict()  # Store errors for all methods and τ values

for (i, τ) in enumerate(τs)
    # Initialize the error storage for this specific τ
    errors_data[τ] = Dict(method => Float64[] for method in Mthds)

    # Reference solution by Petviashvili algorithm
    λ = 1/τ; N_ref = 2^11
    x0, u0 = petviashvili_kdvh(; xmin = xmin, xmax = xmax, λ = λ, c = c, N = N_ref, maxiter = 1000, tol = tols[i]);

    #u0itp = LinearInterpolation((x,), u0, extrapolation_bc = Periodic())
    u0itp = CubicSplineInterpolation((x0,), u0, extrapolation_bc = Periodic())

    function u_traveling(t, x::Number, equation)
        x_t = mod(x - c * t - xmin, xmax - xmin) + xmin
        u0itp(x_t)
    end
    function u_traveling(t, x::AbstractVector, equation)
        u_traveling.(t, x, equation)
    end

    # Iterate over methods
    for alg in Mthds
        # Time steps and vector to store errors
        dts = DTs[i,:]; errs_wrt_Pet_sol = Float64[]

        for dt in dts
            equation = HyperbolizedKdV(τ)
            (; q0, parameters) = setup(u_traveling, equation, tspan, D1)

            @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                                   rhs_nonstiff!,
                                   q0, tspan, parameters, alg;
                                   dt = dt)

            x = grid(parameters.D1)
            u = get_u(sol.u[end], equation)
            u_ref_Pet = u_traveling(tspan[end], x, HyperbolizedKdV(τ))

            err_wrt_Pet_u = integrate(abs2, u - u_ref_Pet, parameters.D1) |> sqrt
            push!(errs_wrt_Pet_sol, err_wrt_Pet_u)
        end
        # Store the errors for the current τ and the method alg
        errors_data[τ][alg] = errs_wrt_Pet_sol;
    end
end


In [ ]:
MthdNames = ["SSP2-ImEx(2,2,2)", "SSP3-ImEx(3,4,3)", "ARS(2,2,2)", "ARK3(2)4L[2]SA"]
# Create the dictionary
MthdDict = Dict(zip(Mthds, MthdNames))
markers = [:circle, :rect, :diamond, :utriangle]

# Figure info
yl = [1.0e-3,1.0e-9,1.0e-9,1.0e-9]; yu = [1.0e-1,1.0e-1,1.0e-1,1.0e-1]
P = [2,3]; ST = [3 3; 2 2; 3 3; 3 3]; EN = [7 6; 6 6; 7 7; 7 7]
Fac = [3.5e-2 6.0e-3; 7.0e-2 1.0e-3; 7.0e-2 1.0e-3; 7.0e-2 1.0e-3]

fig_kdvh_AA = Figure(size = (1000, 900))
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for (i, τ) in enumerate(τs)
    dts = DTs[i, :]
    τ_str = τ == 1.0 ? L"\tau = 1" : L"\tau = 10^{-%$(Int(-log10(τ)))}"
    # Create a new axis for each value of τ
    ax = Axis(fig_kdvh_AA[positions[i]...];
              xlabel = L"\Delta t", ylabel = L"Error of $u$", title = τ_str,
              xscale = log10, yscale = log10)

    # Loop over methods and plot the error data for each method
    for (j, alg) in enumerate(Mthds)
        scatterlines!(ax, dts, errors_data[τ][alg]; label = MthdDict[alg], marker = markers[j])
    end

    # Add the reference scaling line after plotting the methods' data
    lines!(ax, dts[ST[i,1]:EN[i,1]], Fac[i,1] * dts[ST[i,1]:EN[i,1]].^(P[1]);
           label = L"\mathcal{O}(\Delta t^{%$(P[1])})", linestyle = :dash)
    lines!(ax, dts[ST[i,2]:EN[i,2]], Fac[i,2] * dts[ST[i,2]:EN[i,2]].^(P[2]);
           label = L"\mathcal{O}(\Delta t^{%$(P[2])})", linestyle = :dot)

    # Show the legend
    axislegend(ax, position = :lt, framevisible = false)
    ylims!(ax, yl[i], yu[i])
end

save(joinpath(save_path_1, "error_convergence_τ_all.pdf"), fig_kdvh_AA)
fig_kdvh_AA

In [ ]:
tspan

### Error growth plots for different methods, with and without RK relaxation, applied to $\text{KdVH}(\tau)$, with the exact solution computed using the Petviashvili method.

In [ ]:
function kdvh_exact_solitary_wave_error_growth(; domain_traversals = 2,
                                                 τ = 1.0e-2,
                                                 xmin = -40.0,
                                                 xmax = 40.0,
                                                 c = 1.2,
                                                 accuracy_order = 8, N = 2^8,
                                                 alg = ARS443(), dt = 0.01,
                                                 kwargs...)


    # Reference solution by Petviashvili algorithm
    λ = 1/τ; N_ref = 2^10
    x, u0 = petviashvili_kdvh(; xmin = xmin, xmax = xmax, λ = λ, c = c, N = N_ref, maxiter = 1000, tol = 1.0e-12)

    u0itp = LinearInterpolation((x,), u0, extrapolation_bc = Periodic())
    function u_traveling(t, x::Number, equation)
        x_t = mod(x - c * t - xmin, xmax - xmin) + xmin
        u0itp(x_t)
    end
    function u_traveling(t, x::AbstractVector, equation)
        u_traveling.(t, x, equation)
    end


    # Initialization of physical and numerical parameters
    tspan = (0.0, domain_traversals * (xmax - xmin) / c)

    D1 = upwind_operators(periodic_derivative_operator;
       derivative_order = 1, accuracy_order,
       xmin, xmax, N)

    # Setup plot
    fig_err = Figure()
    ax = Axis(fig_err[1, 1]; xlabel = L"t", ylabel = L"Error of $u$",
              xscale = log10, yscale = log10)

    # Setup callback computing the error
    series_t = Vector{Float64}()
    series_error = Vector{Float64}()
    callback = let series_t = series_t, series_error = series_error
        function (q, parameters, t)
            (; tmp, equation) = parameters

            u = get_u(q, equation)
            u_ref = u_traveling(t, grid(parameters.D1), HyperbolizedKdV(τ))

            @. tmp = u - u_ref
            err = integrate(abs2, tmp, parameters.D1) |> sqrt

            push!(series_t, t)
            push!(series_error, err)
            return nothing
        end
    end


    # HyperbolizedKdV with IMEX
    let equation = HyperbolizedKdV(τ)
        (; q0, parameters) = setup(u_traveling, HyperbolizedKdV(τ), tspan, D1)

        @info "Hyperbolized KdV without relaxation"
        empty!(series_t)
        empty!(series_error)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt, callback, kwargs...)
        lines!(ax, series_t, series_error;
               label = "KdVH($τ), baseline")

        @info "Hyperbolized KdV with relaxation"
        empty!(series_t)
        empty!(series_error)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt, callback, relaxation = true, kwargs...)
        lines!(ax, series_t, series_error;
               label = "KdVH($τ), relaxation")
    end
    n = length(series_t)
    st_ind1 = round(Int, 0.05 * n)
    st_ind2 = round(Int, 0.2 * n)

    println("st_ind1 = ", st_ind1)
    println("st_ind2 = ", st_ind2)

    # Use the start index to slice the array from 20% to the end
    lines!(ax, series_t[st_ind1:100:end], 1e-3 * series_t[st_ind1:100:end];
           label = L"\mathcal{O}(t)", linestyle = :dash)
    lines!(ax, series_t[st_ind1:100:st_ind2], 4e-4 * series_t[st_ind1:100:st_ind2].^ 2;
           label = L"\mathcal{O}(t^2)", linestyle = :dot)

    axislegend(ax, position = :lt, framevisible = false)

    return fig_err
end

In [ ]:
τ = 1.0e-2; xmin = -40.0; xmax = 40.0; c = 1.2; ImEx_name = "SSP2ImEx222"
fig_kdvh_err = kdvh_exact_solitary_wave_error_growth(; domain_traversals = 5,
                                                  τ = τ,
                                                  xmin = xmin,
                                                  xmax = xmax,
                                                  c = c,
                                                  accuracy_order = 8, N = 2^8,
                                                  alg = SSP2ImEx222(), dt = 0.05)

In [ ]:
# Saving the file
using Printf
save(joinpath(save_path_1, "KdVH_Err_growth_" * ImEx_name * "_τ_$(@sprintf("%.1e", τ))_c_$(@sprintf("%.1f", c)).pdf"), fig_kdvh_err)


In [ ]:
τ = 1.0e-2; xmin = -40.0; xmax = 40.0; c = 1.2; ImEx_name = "ARS222"
fig_kdvh_err = kdvh_exact_solitary_wave_error_growth(; domain_traversals = 5,
                                                  τ = τ,
                                                  xmin = xmin,
                                                  xmax = xmax,
                                                  c = c,
                                                  accuracy_order = 8, N = 2^8,
                                                  alg = ARS222(), dt = 0.05)

In [ ]:
# Saving the file
save(joinpath(save_path_1, "KdVH_Err_growth_" * ImEx_name * "_τ_$(@sprintf("%.1e", τ))_c_$(@sprintf("%.1f", c)).pdf"), fig_kdvh_err)


### RK relaxation on $\text{KdVH}(\tau)$ to verify linear and quadratic error convergence using conservative versus non-conservative methods.

The error growth of the solutions for the KdV and $\text{KdVH}(\tau)$ systems is analyzed by comparing conservative methods using RK relaxation with non-conservative methods. This approach demonstrates linear error convergence for conservative methods and quadratic error convergence for non-conservative methods. In each case, the numerical solution is compared to the analytical solution of the KdV equation.

In the figure, we will observe:

- Error of the time-stepping method without relaxation for the KdV solution with respect to the KdV analytical solution.
- Error of the time-stepping method with relaxation for the KdV solution with respect to the KdV analytical solution.
- Error of the time-stepping method without relaxation for the KdVH solution (with a particular $\tau$) with respect to the KdV analytical solution.
- Error of the time-stepping method with relaxation for the KdVH solution (with a particular $\tau$) with respect to the KdV analytical solution.

In [ ]:
function kdvh_kdv_solitary_wave_error_growth(; τ = 1.0e-4,
                                              domain_traversals = 10,
                                              accuracy_order = 8, N = 2^10,
                                              alg = ARS443(), dt = 0.025,
                                              kwargs...)
    # Initialization of physical and numerical parameters
    (; xmin, xmax, c) = solitary_wave_setup()
    tspan = (0.0, domain_traversals * (xmax - xmin) / c)

    D1 = upwind_operators(periodic_derivative_operator;
                          derivative_order = 1, accuracy_order,
                          xmin, xmax, N)
        # This provides the D3 aw well in case of KdV equation,  I checked.

    # Setup plot
    fig_err = Figure()
    ax = Axis(fig_err[1, 1]; xlabel = L"t", ylabel = L"Error of $u$",
              xscale = log10, yscale = log10)


    # Setup callback computing the error
    series_t = Vector{Float64}()
    series_error = Vector{Float64}()
    callback = let series_t = series_t, series_error = series_error
        function (q, parameters, t)
            (; tmp, equation) = parameters

            u = get_u(q, equation)
            u_ref = solitary_wave_solution(t, grid(parameters.D1), KdV())

            @. tmp = u - u_ref
            err = integrate(abs2, tmp, parameters.D1) |> sqrt

            push!(series_t, t)
            push!(series_error, err)
            return nothing
        end
    end


    # KdV
    let equation = KdV()
        (; q0, parameters) = setup(solitary_wave_solution, equation,
                                   tspan, D1)

        @info "KdV without relaxation"
        empty!(series_t)
        empty!(series_error)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt, callback, kwargs...)
        lines!(ax, series_t, series_error;
               label = "KdV, baseline")

        @info "KdV with relaxation"
        empty!(series_t)
        empty!(series_error)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt, callback, relaxation = true, kwargs...)
        lines!(ax, series_t, series_error;
               label = "KdV, relaxation")
    end


    # HyperbolizedKdV with IMEX
    let equation = HyperbolizedKdV(τ)
        (; q0, parameters) = setup(solitary_wave_solution, equation,
                                   tspan, D1)

        @info "Hyperbolized KdV without relaxation"
        empty!(series_t)
        empty!(series_error)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt, callback, kwargs...)
        lines!(ax, series_t, series_error;
               label = "KdVH, baseline")

        @info "Hyperbolized KdV with relaxation"
        empty!(series_t)
        empty!(series_error)
        @time sol = solve_imex(rhs_stiff!, operator(rhs_stiff!, parameters),
                               rhs_nonstiff!,
                               q0, tspan, parameters, alg;
                               dt, callback, relaxation = true, kwargs...)
        lines!(ax, series_t, series_error;
               label = "KdVH, relaxation")
    end

    # Use the start index to slice the array from 20% to the end
    n = length(series_t)
    st_ind1 = round(Int, 0.05 * n)
    st_ind2 = round(Int, 0.2 * n)
    lines!(ax, series_t[st_ind1:100:end], 1e-3 * series_t[st_ind1:100:end];
           label = L"\mathcal{O}(t)", linestyle = :dash)
    lines!(ax, series_t[st_ind1:100:st_ind2], 4e-4 * series_t[st_ind1:100:st_ind2].^ 2;
           label = L"\mathcal{O}(t^2)", linestyle = :dot)

    axislegend(ax, position = :lt, framevisible = false)

    return fig_err
end

In [ ]:
τ = 1.0e-3; ImEx_name = "ARS443"
fig_err = kdvh_kdv_solitary_wave_error_growth(; τ = τ,
                                              domain_traversals = 5,
                                              accuracy_order = 8, N = 2^8,
                                              alg = ARS443(), dt = 0.05)


In [ ]:
# Saving the file
save(joinpath(save_path_1, "Err_growth_" * ImEx_name * "_τ_$(@sprintf("%.1e", τ)).pdf"), fig_err)


In [ ]:
τ = 1.0e-4; ImEx_name = "ARS443"
fig_err = kdvh_kdv_solitary_wave_error_growth(; τ = τ,
                                              domain_traversals = 5,
                                              accuracy_order = 8, N = 2^8,
                                              alg = ARS443(), dt = 0.05)


In [ ]:
# Saving the file
save(joinpath(save_path_1, "Err_growth_" * ImEx_name * "_τ_$(@sprintf("%.1e", τ)).pdf"), fig_err)


In [ ]:
τ = 1.0e-5; ImEx_name = "ARS443"
fig_err = kdvh_kdv_solitary_wave_error_growth(; τ = τ,
                                              domain_traversals = 5,
                                              accuracy_order = 8, N = 2^8,
                                              alg = ARS443(), dt = 0.05)


In [ ]:
# Saving the file
save(joinpath(save_path_1, "Err_growth_" * ImEx_name * "_τ_$(@sprintf("%.1e", τ)).pdf"), fig_err)


In [ ]:
τ = 1.0e-6; ImEx_name = "ARS443"
fig_err = kdvh_kdv_solitary_wave_error_growth(; τ = τ,
                                              domain_traversals = 5,
                                              accuracy_order = 8, N = 2^8,
                                              alg = ARS443(), dt = 0.05)


In [ ]:
# Saving the file
save(joinpath(save_path_1, "Err_growth_" * ImEx_name * "_τ_$(@sprintf("%.1e", τ)).pdf"), fig_err)
